# Initialization. Load previous state. Load modules

In [1]:
import dill
# Restore the entire session
#dill.load_session('PTRMS_featsel_class.db')

In [2]:
#check last result on disk
#dir()

In [3]:
import json

import pandas as pd
import numpy as np

from sklearn.model_selection import LeaveOneGroupOut

from xgboost import XGBClassifier,XGBRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor


In [4]:
#fastRFE: RFE own version
#Change only the step value, proportional to remaining features
from sklearn.feature_selection import RFE
from numbers import Integral
from sklearn.base import clone
from sklearn.feature_selection._base import _get_feature_importances

class fastRFE(RFE):
    
    def __init__(
        self,
        estimator,
        *,
        n_features_to_select=None,
        step=0.9,
        verbose=0,
        importance_getter="auto"
    ):
        self.estimator = estimator
        self.n_features_to_select = n_features_to_select
        self.step = step
        self.importance_getter = importance_getter
        self.verbose = verbose
        
    def fit(self, X, y, step_score=None, **fit_params):
        # Parameter step_score controls the calculation of self.step_scores_
        # step_score is not exposed to users and is used when implementing RFECV
        # self.step_scores_ will not be calculated when calling _fit through fit
        tags = self._get_tags()        
        X, y = self._validate_data(
            X,
            y,
            accept_sparse="csc",
            ensure_min_features=2,
            force_all_finite=not tags.get("allow_nan", True),
            multi_output=True,
        )
        
        # Initialization
        n_features = X.shape[1]
        if self.n_features_to_select is None:
            n_features_to_select = n_features // 2
        elif isinstance(self.n_features_to_select, Integral):  # int
            n_features_to_select = self.n_features_to_select
            if n_features_to_select > n_features:
                warnings.warn(
                    (
                        f"Found {n_features_to_select=} > {n_features=}. There will be"
                        " no feature selection and all features will be kept."
                    ),
                    UserWarning,
                )
        else:  # float
            n_features_to_select = int(n_features * self.n_features_to_select)
        
        if 0.0 < self.step < 1.0:
            step = int(max(1, self.step * n_features))
        else:
            step = int(self.step)
        
        support_ = np.ones(n_features, dtype=bool)
        ranking_ = np.ones(n_features, dtype=int)
        
        if step_score:
            self.step_n_features_ = []
            self.step_scores_ = []
        
        # Elimination
        while np.sum(support_) > n_features_to_select:
            # Remaining features
            features = np.arange(n_features)[support_]
            
            # Rank the remaining features
            estimator = clone(self.estimator)
            if self.verbose > 0:
                print("Fitting estimator with %d features." % np.sum(support_))
            
            estimator.fit(X[:, features], y, **fit_params)
            
            # Get importance and rank them
            importances = _get_feature_importances(
                estimator,
                self.importance_getter,
                transform_func="square",
            )
            ranks = np.argsort(importances)
            
            # for sparse case ranks is matrix
            ranks = np.ravel(ranks)
            
            # Eliminate the worse features
            threshold = min(step, np.sum(support_) - n_features_to_select)
            
            # Compute step score on the previous selection iteration
            # because 'estimator' must use features
            # that have not been eliminated yet
            if step_score:
                self.step_n_features_.append(len(features))
                self.step_scores_.append(step_score(estimator, features))
            support_[features[ranks][:threshold]] = False
            ranking_[np.logical_not(support_)] += 1
            ##############################################
            ##############################################
            # Pablo: change step in proportion to remaining features
            if 0.0 < self.step < 1.0:
                step = int(max(1, self.step * np.sum(support_)))
            
            ##############################################
            
        # Set final attributes
        features = np.arange(n_features)[support_]
        self.estimator_ = clone(self.estimator)
        self.estimator_.fit(X[:, features], y, **fit_params)
        
        # Compute step score when only n_features_to_select features left
        if step_score:
            self.step_n_features_.append(len(features))
            self.step_scores_.append(step_score(self.estimator_, features))
        self.n_features_ = support_.sum()
        self.support_ = support_
        self.ranking_ = ranking_
        
        return self

In [5]:

def rfe_logo_feature_selection(IDs, y, X, estimator, param_grid={}, step=0.1, n_features_to_select=1, verbose=True):
    """
    Perform Recursive Feature Elimination (RFE) with Leave-One-Group-Out cross-validation.

    Parameters:
        IDs (pd.Series or np.array): Group identifiers for Leave-One-Group-Out (LOGO).
        y (np.array): Target variable.
        X (pd.DataFrame): Feature matrix.
        estimator (class): Estimator class (e.g., SVM, XGBClassifier).
        param_grid (dict): Dictionary of hyperparameters to initialize the estimator.
        step (float or int): Number of features to remove per iteration in RFE.
        n_features_to_select(int): Stop RFE when this point is rechead.
        verbose (bool): If True, print progress messages.

    Returns:
        dict: A dictionary with group labels as keys and feature ranking lists as values.
    """

    logo = LeaveOneGroupOut()
    unique_groups = np.unique(IDs)
    feature_rankings = {}

    for i, (train_idx, test_idx) in enumerate(logo.split(X, y, groups=IDs)):
        group_label = unique_groups[i]  # Get the group label
        
        if verbose:
            print(f"Processing group {group_label} ({i+1}/{len(unique_groups)})...")

        # Split data
        X_train, y_train = X.iloc[train_idx], y[train_idx]

        # Initialize the estimator with given parameters
        model = estimator(**param_grid)

        # Apply RFE
        rfe = fastRFE(model, step=step,n_features_to_select=n_features_to_select)
        rfe.fit(X_train, y_train)

        # Store feature ranking
        feature_rankings[group_label] = rfe.ranking_.tolist()

    return feature_rankings


In [6]:
def remove_highly_correlated_features(X, threshold=0.95):
    """
    Removes columns in X that have a linear correlation higher than 'threshold'
    with any other column, keeping the column with the lowest index.
    
    Parameters:
    X (pd.DataFrame): The input dataset.
    threshold (float): The correlation threshold to remove features.
    
    Returns:
    pd.DataFrame: The dataset with highly correlated features removed.
    """
    corr_matrix = X.corr().abs()  # Compute absolute correlation matrix
    upper_tri = np.triu(corr_matrix, k=1)  # Upper triangle of correlation matrix (excluding diagonal)
    
    to_remove = set()  # Track columns to remove
    for i in range(len(X.columns)):
        for j in range(i + 1, len(X.columns)):
            if upper_tri[i, j] > threshold:
                to_remove.add(X.columns[j])  # Remove the one with higher index
                
    return X.drop(columns=to_remove)


# Load datasets froms json file

In [7]:
# Load data from JSON
with open('data_classif.json', 'r') as json_file:
    loaded_datasets = json.load(json_file)


# Run feat sel

## All datasets

In [26]:
datasets = ["Tea", "Gum2", "Gum3", "Cafe", "Ham", "Pesce", "Spinaci", "Peperoncini", "Funghi13", "Funghi20", "Funghi21","Lab"]

# Dictionary to store results
results = {dataset: {} for dataset in datasets}

# Define model parameters
models = {
    "XGBoost_full": {
        "estimator": XGBClassifier,
        "param_grid": {
            'device': 'cpu',
            'n_estimators': 1000, 
            'eta': 0.12, 
            'max_depth': 3, 
            'subsample': 1, 
            'colsample_bytree': 1
        }
    },
    "XGBoost_025": {
        "estimator": XGBClassifier,
        "param_grid": {
            'device': 'cpu',
            'n_estimators': 1000, 
            'eta': 0.12, 
            'max_depth': 3, 
            'subsample': 0.8, 
            'colsample_bytree': 0.75
        }
    },
    "RF": {
        "estimator": RandomForestClassifier,
        "param_grid": {
            'n_estimators': 1000
        },
    }
}

# Loop through datasets
for dataset in datasets:
    this_data = loaded_datasets[dataset]
    
    IDs = pd.Series(this_data["IDs"])
    y = np.array(this_data["y"])
    X = pd.DataFrame(this_data["X"])

    #Remove correlated features before selection
    X = remove_highly_correlated_features(X, threshold=0.95)
    
    for model_name, model_info in models.items():
        feature_ranks = rfe_logo_feature_selection(
            IDs=IDs, 
            y=y, 
            X=X, 
            estimator=model_info["estimator"], 
            param_grid=model_info["param_grid"], 
            verbose=False
        )
        results[dataset][model_name] = feature_ranks
        print(f"{dataset}-{model_name}: {feature_ranks}")

Cafe-XGBoost_full: {'P_BRA_1': [45, 45, 45, 8, 45, 45, 45, 45, 45, 45, 45, 16, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 44, 44, 44, 44, 44, 44, 44, 44, 6, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 43, 43, 44, 44, 44, 2, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 42, 3, 42, 42, 1, 42, 43, 43, 42, 42, 43, 42, 42, 42, 42, 42, 42, 17, 42, 42, 42, 42, 41, 41, 41, 41, 41, 42, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 40, 40, 40, 40, 40, 40, 40, 40, 40, 39, 39, 5, 40, 40, 40, 40, 40, 39, 39, 39, 39, 39, 39, 38, 38, 38, 39, 39, 39, 39, 39, 38, 38, 38, 38, 38, 37, 37, 37, 37, 37, 38, 38, 38, 37, 37, 37, 36, 36, 36, 36, 36, 36, 37, 37, 36, 36, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 47, 47, 47, 47, 47, 10, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 48, 48, 48, 48, 48, 48, 48, 4, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 46, 46, 9, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 

In [33]:
from itertools import combinations

# get the k best ranked features
def top_k_features(ranking, k=2):
    return sorted(range(len(ranking)), key=lambda i: ranking[i])[:k]

# Computes Rank-Biased Overlap (RBO) for two ranked lists S and T.
def rbo(S, T, p=0.9):
    k = min(len(S), len(T))  # Limit to the minimum length
    rbo_score = 0.0
    
    for d in range(1, k + 1):  # Depth from 1 to k
        overlap = len(set(S[:d]) & set(T[:d])) / d
        rbo_score += (overlap * (p ** d))
    
    return (1 - p) * rbo_score


def mean_jackard_same_model(datasets,models,k):
    mean_jackard = {dataset: {} for dataset in datasets}
    
    for dataset in datasets:
        for model_name, model_info in models.items():
            rankings=results[dataset][model_name]
            jaccard_values = []
            # Calcular Jaccard para cada par de listas
            for (name1, ranks1), (name2, ranks2) in combinations(rankings.items(), 2):
                top1 = top_k_features(ranks1,k)
                top2 = top_k_features(ranks2,k)
    
                jaccard_index = len(set(top1) & set(top2)) / len(set(top1) | set(top2))
                jaccard_values.append(jaccard_index)
    
            mean_jackard[dataset][model_name]=sum(jaccard_values) / len(jaccard_values)
            print(f"{dataset}-{model_name}-Average Jaccard Index: {mean_jackard[dataset][model_name]:.4f}")
    return mean_jackard

def mean_rbo_same_model(datasets,models,k):
    mean_rbo = {dataset: {} for dataset in datasets}
    
    for dataset in datasets:
        for model_name, model_info in models.items():
            rankings=results[dataset][model_name]
            rbo_values = []
            # Calcular Jaccard para cada par de listas
            for (name1, ranks1), (name2, ranks2) in combinations(rankings.items(), 2):
                top1 = top_k_features(ranks1,k)
                top2 = top_k_features(ranks2,k)
    
                rbo_index = rbo(top1, top2, p=0.9)
                rbo_values.append(rbo_index)
    
            mean_rbo[dataset][model_name]=sum(rbo_values) / len(rbo_values)
            print(f"{dataset}-{model_name}-RBO Index: {mean_rbo[dataset][model_name]:.4f}")
    return mean_rbo

def print_as_table(datasets,models,mean_jackard):
    # Print header
    print("Dataset".ljust(12) + " | " + " | ".join(model_name.ljust(15) for model_name in models))

    # Print separator line
    print("-" * (12 + 3 + len(models) * 18))

    # Print results
    for dataset in datasets:
        result_line = f"{dataset.ljust(12)} | " + " | ".join(f"{round(mean_jackard[dataset][model_name],4):<15}" for model_name in models)
        print(result_line)
        

In [34]:
k=5
mean_jac=mean_jackard_same_model(datasets,models,k)
print("")
print("Mean Jackard results for first",k,"features")
print("")
print_as_table(datasets,models,mean_jac)

Cafe-XGBoost_full-Average Jaccard Index: 0.3212
Cafe-XGBoost_025-Average Jaccard Index: 0.3024
Cafe-RF-Average Jaccard Index: 0.1297
Ham-XGBoost_full-Average Jaccard Index: 0.2422
Ham-XGBoost_025-Average Jaccard Index: 0.3764
Ham-RF-Average Jaccard Index: 0.0909

Mean Jackard results for first 5 features

Dataset      | XGBoost_full    | XGBoost_025     | RF             
---------------------------------------------------------------------
Cafe         | 0.3212          | 0.3024          | 0.1297         
Ham          | 0.2422          | 0.3764          | 0.0909         


In [35]:
k=5
mean_rbo=mean_rbo_same_model(datasets,models,k)
print("")
print("Mean RBO results for first",k,"features")
print("")
print_as_table(datasets,models,mean_rbo)

Cafe-XGBoost_full-RBO Index: 0.1172
Cafe-XGBoost_025-RBO Index: 0.1357
Cafe-RF-RBO Index: 0.0547
Ham-XGBoost_full-RBO Index: 0.1084
Ham-XGBoost_025-RBO Index: 0.1488
Ham-RF-RBO Index: 0.0402

Mean RBO results for first 5 features

Dataset      | XGBoost_full    | XGBoost_025     | RF             
---------------------------------------------------------------------
Cafe         | 0.1172          | 0.1357          | 0.0547         
Ham          | 0.1084          | 0.1488          | 0.0402         


In [36]:
k=10
mean_jac=mean_jackard_same_model(datasets,models,k)
print("")
print("Mean Jackard results for first",k,"features")
print("")
print_as_table(datasets,models,mean_jac)

Cafe-XGBoost_full-Average Jaccard Index: 0.3864
Cafe-XGBoost_025-Average Jaccard Index: 0.3404
Cafe-RF-Average Jaccard Index: 0.1867
Ham-XGBoost_full-Average Jaccard Index: 0.3766
Ham-XGBoost_025-Average Jaccard Index: 0.4571
Ham-RF-Average Jaccard Index: 0.1191

Mean Jackard results for first 10 features

Dataset      | XGBoost_full    | XGBoost_025     | RF             
---------------------------------------------------------------------
Cafe         | 0.3864          | 0.3404          | 0.1867         
Ham          | 0.3766          | 0.4571          | 0.1191         


In [37]:
k=10
mean_rbo=mean_rbo_same_model(datasets,models,k)
print("")
print("Mean RBO results for first",k,"features")
print("")
print_as_table(datasets,models,mean_rbo)

Cafe-XGBoost_full-RBO Index: 0.2386
Cafe-XGBoost_025-RBO Index: 0.2395
Cafe-RF-RBO Index: 0.1168
Ham-XGBoost_full-RBO Index: 0.2116
Ham-XGBoost_025-RBO Index: 0.2793
Ham-RF-RBO Index: 0.0816

Mean RBO results for first 10 features

Dataset      | XGBoost_full    | XGBoost_025     | RF             
---------------------------------------------------------------------
Cafe         | 0.2386          | 0.2395          | 0.1168         
Ham          | 0.2116          | 0.2793          | 0.0816         


In [38]:
k=20
mean_jac=mean_jackard_same_model(datasets,models,k)
print("")
print("Mean Jackard results for first",k,"features")
print("")
print_as_table(datasets,models,mean_jac)

Cafe-XGBoost_full-Average Jaccard Index: 0.3511
Cafe-XGBoost_025-Average Jaccard Index: 0.2791
Cafe-RF-Average Jaccard Index: 0.2215
Ham-XGBoost_full-Average Jaccard Index: 0.3355
Ham-XGBoost_025-Average Jaccard Index: 0.2865
Ham-RF-Average Jaccard Index: 0.1497

Mean Jackard results for first 20 features

Dataset      | XGBoost_full    | XGBoost_025     | RF             
---------------------------------------------------------------------
Cafe         | 0.3511          | 0.2791          | 0.2215         
Ham          | 0.3355          | 0.2865          | 0.1497         


In [39]:
k=20
mean_rbo=mean_rbo_same_model(datasets,models,k)
print("")
print("Mean RBO results for first",k,"features")
print("")
print_as_table(datasets,models,mean_rbo)

Cafe-XGBoost_full-RBO Index: 0.3455
Cafe-XGBoost_025-RBO Index: 0.3376
Cafe-RF-RBO Index: 0.1791
Ham-XGBoost_full-RBO Index: 0.3193
Ham-XGBoost_025-RBO Index: 0.3867
Ham-RF-RBO Index: 0.1264

Mean RBO results for first 20 features

Dataset      | XGBoost_full    | XGBoost_025     | RF             
---------------------------------------------------------------------
Cafe         | 0.3455          | 0.3376          | 0.1791         
Ham          | 0.3193          | 0.3867          | 0.1264         


In [11]:
dill.dump_session('PTRMS_featsel_class.db')